In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# Show all output for a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [63]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [64]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-11 17:00:00+00:00


In [65]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [66]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-11 12:18:10,707 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:18:10,737 INFO: Initializing external client
2025-05-11 12:18:10,738 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 12:18:11,365 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


In [67]:
df= query.read()
df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.37s) 


,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00
3,6140.05,1.0,2025-05-11 15:00:00+00:00
4,5905.14,0.0,2025-05-11 15:00:00+00:00
5,5329.03,9.0,2025-05-11 15:00:00+00:00
6,5788.13,33.0,2025-05-11 15:00:00+00:00


In [68]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-11 12:18:13,818 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:18:13,824 INFO: Initializing external client
2025-05-11 12:18:13,825 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 12:18:14,360 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Current UTC time: 2025-05-11 16:18:13.818893+00:00
Next hour: 2025-05-11 17:00:00+00:00
Querying for date range: 2025-05-11 to 2025-05-12
Filtering for hour: 2025-05-11 17:00
Found 0 records


In [69]:
predictions

,start_station_id,predicted_demand,hour


In [70]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [71]:
dt = current_date.ceil('h')

In [72]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.hour > "2025-05-10 05:00:00")  
results = query.read()

2025-05-11 12:18:15,317 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-05-11 12:18:15,322 INFO: Initializing external client
2025-05-11 12:18:15,323 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:18:15,878 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 


In [73]:
results

,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00
3,6140.05,1.0,2025-05-11 15:00:00+00:00
4,5905.14,0.0,2025-05-11 15:00:00+00:00
5,5329.03,9.0,2025-05-11 15:00:00+00:00
6,5788.13,33.0,2025-05-11 15:00:00+00:00


In [74]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [75]:
predictions = fetch_next_hour_predictions()

2025-05-11 12:18:18,278 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:18:18,284 INFO: Initializing external client
2025-05-11 12:18:18,285 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 12:18:18,863 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Current UTC time: 2025-05-11 16:18:18.278813+00:00
Next hour: 2025-05-11 17:00:00+00:00
Found 0 records


In [76]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

2025-05-11 12:18:19,572 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:18:19,577 INFO: Initializing external client
2025-05-11 12:18:19,578 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 12:18:20,107 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


In [77]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 


In [78]:
df["hour"].max()

Timestamp('2025-05-11 15:00:00+0000', tz='Etc/UTC')

In [79]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-11 17:00:00+00:00


In [80]:
df[df["hour"] == next_hour]

,start_station_id,predicted_demand,hour
